In [ ]:
import os
import pandas as pd
from openpyxl import load_workbook

def process_excel_files(root_folder, output_file):
    # 创建一个Excel writer对象，用于写入多个工作表
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        # 遍历根文件夹下的所有子文件夹
        for dataset_name in os.listdir(root_folder):
            subfolder = os.path.join(root_folder, dataset_name)
            
            # 确保是文件夹
            if os.path.isdir(subfolder):
                # 构建Excel文件路径
                excel_file = os.path.join(subfolder, f"{dataset_name}.xlsx")
                
                # 检查Excel文件是否存在
                if os.path.isfile(excel_file):
                    try:
                        # 读取Excel文件
                        df = pd.read_excel(excel_file)
                        
                        # 检查所需的列是否存在
                        if all(col in df.columns for col in ['Gmean', 'MAUC']):
                            # 只保留需要的列
                            df_subset = df[['Gmean', 'MAUC']]
                            
                            # 将数据写入新的工作表
                            df_subset.to_excel(writer, sheet_name=dataset_name, index=False)
                            print(f"成功处理: {dataset_name}")
                        else:
                            print(f"警告: {dataset_name} 文件中缺少所需的列")
                    except Exception as e:
                        print(f"处理 {dataset_name} 时出错: {str(e)}")
                else:
                    print(f"警告: {dataset_name} 文件夹中没有找到对应的Excel文件")

if __name__ == "__main__":
    # 设置根文件夹路径和输出文件路径
    root_folder = "C:\\Users\\zsc\\Desktop\\Ablation study2\\Objective1"  # 替换为您的根文件夹路径
    output_file = "C:\\Users\\zsc\\Desktop\\Ablation study2\\combined_results.xlsx"     # 输出文件名
    
    # 处理所有Excel文件
    process_excel_files(root_folder, output_file)
    
    print("处理完成！结果已保存到", output_file)

## 处理数据

In [7]:
import os
import pandas as pd
import numpy as np
from openpyxl import load_workbook

def process_excel_files(root_folder, combined_output, stats_output):
    # 创建统计数据DataFrame
    stats_data = []
    
    # 创建一个Excel writer对象，用于写入多个工作表
    with pd.ExcelWriter(combined_output, engine='openpyxl') as writer:
        # 遍历根文件夹下的所有子文件夹
        for dataset_name in os.listdir(root_folder):
            subfolder = os.path.join(root_folder, dataset_name)
            
            # 确保是文件夹
            if os.path.isdir(subfolder):
                # 构建Excel文件路径
                excel_file = os.path.join(subfolder, f"{dataset_name}.xlsx")
                
                # 检查Excel文件是否存在
                if os.path.isfile(excel_file):
                    try:
                        # 读取Excel文件
                        df = pd.read_excel(excel_file)
                        
                        # 检查所需的列是否存在
                        if all(col in df.columns for col in ['Gmean', 'MAUC']):
                            # 配对两列数据
                            paired_data = df[['Gmean', 'MAUC']]
                            
                            # 按Gmeans升序排序
                            sorted_data = paired_data.sort_values(by='Gmean', ascending=True)
                            
                            # 取前30对数据
                            top_30 = sorted_data.head(30)
                            
                            # 将数据写入新的工作表
                            top_30.to_excel(writer, sheet_name=dataset_name[:31], index=False)
                            
                            # 计算统计量
                            avg_gmean = np.mean(top_30['Gmean'])
                            avg_mauc = np.mean(top_30['MAUC'])
                            std_gmean = np.std(top_30['Gmean'])
                            std_mauc = np.std(top_30['MAUC'])
                            
                            # 添加到统计数据
                            stats_data.append({
                                'Dataset': dataset_name,
                                'Type': 'AVG',
                                'Gmean': avg_gmean,
                                'MAUC': avg_mauc
                            })
                            stats_data.append({
                                'Dataset': dataset_name,
                                'Type': 'STD',
                                'Gmean': std_gmean,
                                'MAUC': std_mauc
                            })
                            
                            print(f"成功处理: {dataset_name}")
                        else:
                            print(f"警告: {dataset_name} 文件中缺少所需的列")
                    except Exception as e:
                        print(f"处理 {dataset_name} 时出错: {str(e)}")
                else:
                    print(f"警告: {dataset_name} 文件夹中没有找到对应的Excel文件")
    
    # 将统计数据写入另一个Excel文件
    if stats_data:
        stats_df = pd.DataFrame(stats_data)
        # 重新排列数据格式
        final_stats = pd.pivot_table(stats_df, 
                                   index='Dataset', 
                                   columns='Type', 
                                   values=['Gmean', 'MAUC'])
        # 扁平化多级列索引
        final_stats.columns = [f"{col[1]}_{col[0]}" for col in final_stats.columns]
        final_stats.reset_index(inplace=True)
        
        # 按照要求的格式重新组织数据
        formatted_stats = pd.DataFrame()
        for dataset in final_stats['Dataset'].unique():
            dataset_data = final_stats[final_stats['Dataset'] == dataset]
            formatted_stats = pd.concat([
                formatted_stats,
                pd.DataFrame({
                    'Dataset': [dataset, dataset],
                    'Statistic': ['AVG', 'STD'],
                    'Gmean': [dataset_data['AVG_Gmean'].values[0], dataset_data['STD_Gmean'].values[0]],
                    'MAUC': [dataset_data['AVG_MAUC'].values[0], dataset_data['STD_MAUC'].values[0]]
                })
            ])
        
        # 写入统计文件
        with pd.ExcelWriter(stats_output, engine='openpyxl') as stats_writer:
            formatted_stats.to_excel(stats_writer, index=False, sheet_name='Statistics')

if __name__ == "__main__":
     # 设置根文件夹路径和输出文件路径
    root_folder = "C:\\Users\\zsc\\Desktop\\Ablation study2\\Objective1"  # 替换为您的根文件夹路径
    combined_output = "C:\\Users\\zsc\\Desktop\\Ablation study2\\Objective1.xlsx"  # 组合结果文件名
    stats_output = "C:\\Users\\zsc\\Desktop\\Ablation study2\\Statistics_Objective1.xlsx"   # 统计结果文件名
    
    # 处理所有Excel文件
    process_excel_files(root_folder, combined_output, stats_output)
    
    print("处理完成！")
    print(f"组合结果已保存到: {combined_output}")
    print(f"统计结果已保存到: {stats_output}")

成功处理: Automobile
成功处理: Balance_Scale
成功处理: Car
成功处理: Contraceptive
成功处理: Dermatology
成功处理: Ecoli
成功处理: German
成功处理: Nursery
成功处理: Ovarian
成功处理: Page_Blocks
成功处理: Penbased
成功处理: Pen_Digits
成功处理: Satellite
成功处理: Shuttle
成功处理: Splice
成功处理: USPS
成功处理: WallRobot
成功处理: Wine
处理完成！
组合结果已保存到: C:\Users\zsc\Desktop\Ablation study2\Objective1.xlsx
统计结果已保存到: C:\Users\zsc\Desktop\Ablation study2\Statistics_Objective1.xlsx


\begin{tabular}{lcccccc}
\hline
Dataset & Gmean\_1 & Gmean\_2 & Gmean\_3 & MAUC\_1 & MAUC\_2 & MAUC\_3 \\ \hline
Automobile & 70.55(2.66e-2) & 70.09(2.88e-2) & 70.63(3.05e-2) & 93.24(9.39e-3) & 92.95(9.67e-3) & 93.12(9.21e-3) \\
Balance\_Scale & 96.79(8.66e-3) & 97.48(1.07e-2) & 97.34(1.62e-2) & 99.86(8.27e-4) & 99.87(7.23e-4) & 99.87(9.50e-4) \\
Car & 95.44(8.91e-3) & 94.52(8.60e-3) & 94.78(9.41e-3) & 99.79(7.63e-4) & 99.73(6.60e-4) & 99.75(8.79e-4) \\
Contraceptive & 52.88(9.14e-3) & 51.69(1.32e-2) & 51.07(1.51e-2) & 73.90(4.14e-3) & 73.69(5.06e-3) & 73.77(5.56e-3) \\
Dermatology & 92.73(1.53e-2) & 92.34(1.86e-2) & 90.75(2.06e-2) & 98.98(2.55e-3) & 99.01(2.41e-3) & 98.79(2.37e-3) \\
Ecoli & 86.26(1.01e-2) & 85.98(1.73e-2) & 85.60(1.29e-2) & 97.43(3.24e-3) & 97.33(2.97e-3) & 97.42(2.79e-3) \\
German & 66.09(1.48e-2) & 65.02(1.25e-2) & 63.86(1.52e-2) & 75.99(1.03e-2) & 75.74(9.22e-3) & 75.90(1.19e-2) \\
Nursery & 96.17(6.48e-3) & 96.99(8.69e-3) & 96.17(8.85e-3) & 99.85(3.33e-4) & 99.80(4.15e-4) & 99.85(2.82e-4) \\
Ovarian & 97.79(1.16e-2) & 93.35(1.28e-2) & 94.99(2.00e-2) & 99.75(1.02e-3) & 98.37(3.75e-3) & 98.47(3.97e-3) \\
Page\_Blocks & 81.54(8.25e-3) & 79.82(1.02e-2) & 79.57(9.03e-3) & 97.88(2.42e-3) & 97.61(2.97e-3) & 97.63(2.27e-3) \\
Pen\_Digits & 99.45(1.79e-3) & 99.45(1.30e-3) & 99.40(1.41e-3) & 100.00(3.47e-6) & 100.00(2.58e-6) & 100.00(3.05e-6) \\
Penbased & 97.00(5.42e-3) & 96.89(5.41e-3) & 97.01(6.05e-3) & 99.87(4.19e-4) & 99.88(4.85e-4) & 99.86(4.34e-4) \\
Satellite & 87.24(2.42e-3) & 87.25(2.66e-3) & 87.09(3.03e-3) & 98.49(3.30e-4) & 98.49(3.88e-4) & 98.47(3.83e-4) \\
Shuttle & 100.00(0.00e-0) & 100.00(0.00e-0) & 100.00(0.00e-0) & 100.00(0.00e-0) & 100.00(0.00e-0) & 100.00(0.00e-0) \\
Splice & 87.52(7.49e-3) & 87.61(8.07e-3) & 87.69(5.37e-3) & 97.03(2.60e-3) & 97.24(2.39e-3) & 97.32(1.56e-3) \\
USPS & 96.14(1.13e-3) & 96.11(1.73e-3) & 96.10(1.29e-3) & 99.85(1.51e-4) & 99.84(1.36e-4) & 99.85(1.10e-4) \\
WallRobot & 86.77(1.63e-2) & 86.11(1.90e-2) & 86.42(3.06e-2) & 97.68(4.30e-3) & 97.14(5.86e-3) & 97.49(8.91e-3) \\
Wine & 99.78(5.48e-3) & 99.19(1.09e-2) & 99.35(8.99e-3) & 100.00(0.00e-0) & 99.99(3.49e-4) & 99.99(3.49e-4) \\ \hline
\end{tabular}

In [ ]:
import pandas as pd
import os

def merge_algorithm_results(algorithm_files, output_gmean_file, output_mauc_file):
    """
    合并多个算法Excel文件中相同sheet的Gmean和MAUC数据
    
    参数:
    algorithm_files: 算法Excel文件路径列表
    output_gmean_file: 输出的Gmean合并文件路径
    output_mauc_file: 输出的MAUC合并文件路径
    """
    
    # 获取所有sheet名称(假设所有文件的sheet名称和顺序相同)
    with pd.ExcelFile(algorithm_files[0]) as xls:
        sheet_names = xls.sheet_names
    
    # 创建Excel写入对象
    writer_gmean = pd.ExcelWriter(output_gmean_file, engine='openpyxl')
    writer_mauc = pd.ExcelWriter(output_mauc_file, engine='openpyxl')
    
    for sheet in sheet_names:
        # 初始化合并后的DataFrame
        merged_gmean = pd.DataFrame()
        merged_mauc = pd.DataFrame()
        
        for i, file in enumerate(algorithm_files, 1):
            # 读取每个文件中的当前sheet
            df = pd.read_excel(file, sheet_name=sheet)
            
            # 添加算法名称作为列名后缀
            gmean_col = f'算法{i}_Gmean'
            mauc_col = f'算法{i}_MAUC'
            
            # 将数据添加到合并DataFrame
            merged_gmean[gmean_col] = df['Gmean']
            merged_mauc[mauc_col] = df['MAUC']
        
        # 将合并后的数据写入对应的sheet
        merged_gmean.to_excel(writer_gmean, sheet_name=sheet, index=False)
        merged_mauc.to_excel(writer_mauc, sheet_name=sheet, index=False)
    
    # 保存Excel文件
    writer_gmean.close()
    writer_mauc.close()

# 使用示例
if __name__ == "__main__":
    # 替换为您的实际文件路径
    algorithm_files = [
        'C:\\Users\\zsc\\Desktop\\finall_result\\Objective1.xlsx',
        'C:\\Users\\zsc\\Desktop\\finall_result\\Objective2.xlsx',
        'C:\\Users\\zsc\\Desktop\\finall_result\\Objective3.xlsx'
    ]
    
    output_gmean = 'C:\\Users\\zsc\\Desktop\\finall_result\\合并后的Gmean结果.xlsx'
    output_mauc = 'C:\\Users\\zsc\\Desktop\\finall_result\\合并后的MAUC结果.xlsx'
    
    merge_algorithm_results(algorithm_files, output_gmean, output_mauc)
    
    print(f"合并完成！Gmean结果已保存到: {output_gmean}")
    print(f"合并完成！MAUC结果已保存到: {output_mauc}")

In [8]:
import pandas as pd
import os

def merge_algorithm_results(algorithm_files, output_gmean_file, output_mauc_file):
    """
    合并多个算法Excel文件中相同sheet的Gmean和MAUC数据
    
    参数:
    algorithm_files: 算法Excel文件路径列表
    output_gmean_file: 输出的Gmean合并文件路径
    output_mauc_file: 输出的MAUC合并文件路径
    """
    
    # 获取所有sheet名称(假设所有文件的sheet名称和顺序相同)
    with pd.ExcelFile(algorithm_files[0]) as xls:
        sheet_names = xls.sheet_names
    
    # 创建Excel写入对象
    writer_gmean = pd.ExcelWriter(output_gmean_file, engine='openpyxl')
    writer_mauc = pd.ExcelWriter(output_mauc_file, engine='openpyxl')
    
    for sheet in sheet_names:
        # 初始化合并后的DataFrame
        merged_gmean = pd.DataFrame()
        merged_mauc = pd.DataFrame()
        
        for i, file in enumerate(algorithm_files, 1):
            # 读取每个文件中的当前sheet
            df = pd.read_excel(file, sheet_name=sheet)
            
            # 使用您指定的列名格式
            gmean_col = f'algorithm_{i}'
            mauc_col = f'algorithm_{i}'
            
            # 将数据添加到合并DataFrame
            merged_gmean[gmean_col] = df['Gmean']
            merged_mauc[mauc_col] = df['MAUC']
        
        # 将合并后的数据写入对应的sheet
        merged_gmean.to_excel(writer_gmean, sheet_name=sheet, index=False)
        merged_mauc.to_excel(writer_mauc, sheet_name=sheet, index=False)
    
    # 保存Excel文件
    writer_gmean.close()
    writer_mauc.close()

# 使用示例
if __name__ == "__main__":
    # 替换为您的实际文件路径
    algorithm_files = [
        'C:\\Users\\zsc\\Desktop\\finall_result\\Objective1.xlsx',
        'C:\\Users\\zsc\\Desktop\\finall_result\\Objective2.xlsx',
        'C:\\Users\\zsc\\Desktop\\finall_result\\Objective3.xlsx'
    ]
    
    output_gmean = 'C:\\Users\\zsc\\Desktop\\finall_result\\合并后的Gmean结果.xlsx'
    output_mauc = 'C:\\Users\\zsc\\Desktop\\finall_result\\合并后的MAUC结果.xlsx'
    
    merge_algorithm_results(algorithm_files, output_gmean, output_mauc)
    
    print(f"合并完成！Gmean结果已保存到: {output_gmean}")
    print(f"合并完成！MAUC结果已保存到: {output_mauc}")

合并完成！Gmean结果已保存到: C:\Users\zsc\Desktop\finall_result\合并后的Gmean结果.xlsx
合并完成！MAUC结果已保存到: C:\Users\zsc\Desktop\finall_result\合并后的MAUC结果.xlsx
